# Hands-on example (Azure ML)

In [1]:
!python --version

Python 3.8.1


In [2]:
import os
import warnings
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Sklearn
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

#Azure
import azureml.core
import azureml.dataprep
from azureml.core import Workspace, Dataset
from azure.storage.blob import BlobServiceClient
from azureml.core.authentication import ServicePrincipalAuthentication

#MLFlow
import mlflow
import mlflow.sklearn
from mlflow.entities import ViewType

#Temporarily filter all warning for demo
warnings.filterwarnings("ignore")

In [3]:
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.15.0


### Update Working Directory

In [5]:
os.getcwd()

'/Users/antfra/Desktop/MLFlow In Azure/notebooks'

In [18]:
#get path to parent folder (root folder)
parent_folder=os.path.dirname(os.getcwd())
parent_folder

'/Users/antfra/Desktop/MLFlow In Azure'

In [19]:
#set system path to parent folder
sys.path.insert(0,parent_folder)

### Set Up Authentication

In [11]:
#get environment variables
from dotenv import load_dotenv
from settings import (AZURE_SUBSCRIPTION_ID,AZURE_RESOURCE_GROUP, 
                     AZURE_TENANT_ID)

In [48]:
subscription_id = AZURE_SUBSCRIPTION_ID
resource_group = AZURE_RESOURCE_GROUP
workspace_name = 'mlflow_tutorial' #previous existing azureml workspace

## 0. The data

* The data set used in this example is from http://archive.ics.uci.edu/ml/datasets/Wine+Quality
* P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis.
* Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009.

#### Get workspace object from azureml sdk

In [49]:
##Use this if inside AML Studio
# ws = Workspace.from_config()
ws = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(ws, name='wine-quality')
dataset.to_pandas_dataframe().head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


## 1. Tracking experiments

### Set Tracking URI to Azure ML Workspace

In [21]:
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

In [50]:
mlflow.tracking.get_tracking_uri()

'azureml://eastus.experiments.azureml.net/mlflow/v1.0/subscriptions/e1eb783d-78a5-42a4-bae3-bc0ddd433898/resourceGroups/robotdemo/providers/Microsoft.MachineLearningServices/workspaces/mlflow_tutorial?'

In [55]:
exp_name = "My Simple New Experiment AML"

In [ ]:
mlflow.create_experiment(exp_name)

In [57]:

mlflow.set_experiment(exp_name)

#### List experiments

In [58]:
mlflow.tracking.MlflowClient().list_experiments()

[<Experiment: artifact_location='', experiment_id='fe3ff3a4-0fa3-4075-bd4f-8352d3b501d9', lifecycle_stage='active', name='ElasticNet_wine_AML', tags={}>,
 <Experiment: artifact_location='', experiment_id='fc950002-c0b7-4ca0-b1dc-d1685a7ab854', lifecycle_stage='active', name='My Simple New Experiment AML', tags={}>]

### What do we track?

- **Code Version**: Git commit hash used for the run (if it was run from an MLflow Project)
- **Start & End Time**: Start and end time of the run
- **Source**: what code run?
- **Parameters**: Key-value input parameters.
- **Metrics**: Key-value metrics, where the value is numeric (can be updated over the run)
- **Artifacts**: Output files in any format.

In [40]:
def eval_metrics(actual, pred):
    # compute relevant metrics
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


def load_data(dataset):
    data = dataset.to_pandas_dataframe()

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]
    return train_x, train_y, test_x, test_y


def train(alpha=0.5, l1_ratio=0.5,full_view=False):
    # train a model with given parameters
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file (make sure you're running this from the root of MLflow!)
    data_path = "data/wine-quality.csv"
    train_x, train_y, test_x, test_y = load_data(dataset)

    # Useful for multiple runs (only doing one run in this sample notebook)    
    with mlflow.start_run() as run:
        # Execute ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param(key="alpha", value=alpha)
        mlflow.log_param(key="l1_ratio", value=l1_ratio)
        mlflow.log_metric(key="rmse", value=rmse)
        mlflow.log_metrics({"mae": mae, "r2": r2})
        
        #Log model
        mlflow.sklearn.log_model(lr, "model")
        
        #print artifact uri
        print("Save to: {}".format(mlflow.get_artifact_uri()))
        
        #print run_id
        print(f"RunID: {run.info.run_uuid}")
        
        #print experiment id
        print(f"Experiment ID: {run.info.experiment_id}")
        
        if full_view:
            print("Run IDs: \n{}".format(mlflow.search_runs(ViewType.ACTIVE_ONLY)))
        else:
            pass
        
                

In [41]:
train(0.5, 0.5)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.82224284975954
  MAE: 0.6278761410160693
  R2: 0.12678721972772689


2020/10/08 17:03:01 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed, possibly due older server version. The model artifacts have been logged successfully under azureml://experiments/My Simple New Experiment AML/runs/14ec4912-f0f8-42ea-a700-7a8da0a0046b/artifacts. In addition to exporting model artifacts, MLflow clients 1.7.0 and above attempt to record model metadata to the  tracking store. If logging to a mlflow server via REST, consider  upgrading the server version to MLflow 1.7.0 or above.


Save to: azureml://experiments/My Simple New Experiment AML/runs/14ec4912-f0f8-42ea-a700-7a8da0a0046b/artifacts
RunID: 14ec4912-f0f8-42ea-a700-7a8da0a0046b
Experiment ID: fc950002-c0b7-4ca0-b1dc-d1685a7ab854


In [35]:
train(0.2, 0.2)

Elasticnet model (alpha=0.200000, l1_ratio=0.200000):
  RMSE: 0.7859129997062341
  MAE: 0.6155290394093893
  R2: 0.20224631822892103


2020/10/08 16:20:15 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed, possibly due older server version. The model artifacts have been logged successfully under azureml://experiments/My Simple New Experiment AML/runs/42a2682c-6b06-4806-8912-6781aa4a9488/artifacts. In addition to exporting model artifacts, MLflow clients 1.7.0 and above attempt to record model metadata to the  tracking store. If logging to a mlflow server via REST, consider  upgrading the server version to MLflow 1.7.0 or above.


Save to: azureml://experiments/My Simple New Experiment AML/runs/42a2682c-6b06-4806-8912-6781aa4a9488/artifacts
RunID: 42a2682c-6b06-4806-8912-6781aa4a9488
Experiment ID: fc950002-c0b7-4ca0-b1dc-d1685a7ab854


In [24]:
train(0.1, 0.1,full_view=True)

Elasticnet model (alpha=0.100000, l1_ratio=0.100000):
  RMSE: 0.7792546522251949
  MAE: 0.6112547988118587
  R2: 0.2157063843066196
Save to: azureml://experiments/ElasticNet_wine_AML/runs/da3c9d94-7549-4926-8f7c-2b776477d612/artifacts
Run IDs: 
Empty DataFrame
Columns: [run_id, experiment_id, status, artifact_uri, start_time, end_time]
Index: []


2020/10/07 15:22:10 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed, possibly due older server version. The model artifacts have been logged successfully under azureml://experiments/ElasticNet_wine_AML/runs/da3c9d94-7549-4926-8f7c-2b776477d612/artifacts. In addition to exporting model artifacts, MLflow clients 1.7.0 and above attempt to record model metadata to the  tracking store. If logging to a mlflow server via REST, consider  upgrading the server version to MLflow 1.7.0 or above.


## 1.1 Comparing Runs in UI (Azure Portal)
Connect to Azure Portal to view experiments and run metrics.

In [52]:
exp_name

'My Simple New Experiment AML'

In [53]:
ws.experiments

{'ElasticNet_wine_AML': Experiment(Name: ElasticNet_wine_AML,
 Workspace: mlflow_tutorial)}

In [54]:
ws.experiments[exp_name]

Name,Workspace,Report Page,Docs Page
ElasticNet_wine_AML,mlflow_tutorial,Link to Azure Machine Learning studio,Link to Documentation


## 1.2 Comparing Runs Programmatically

### Look at MLFlow Directory Structure

In [62]:
from mlflow.tracking import MlflowClient

list_of_experiments = MlflowClient().list_experiments()
list_of_experiments

[<Experiment: artifact_location='', experiment_id='fe3ff3a4-0fa3-4075-bd4f-8352d3b501d9', lifecycle_stage='active', name='ElasticNet_wine_AML', tags={}>,
 <Experiment: artifact_location='', experiment_id='fc950002-c0b7-4ca0-b1dc-d1685a7ab854', lifecycle_stage='active', name='My Simple New Experiment AML', tags={}>]

In [64]:
#main experiment
exp_id = list_of_experiments[0].experiment_id
exp_id

'fe3ff3a4-0fa3-4075-bd4f-8352d3b501d9'

In [65]:
MlflowClient().get_experiment(exp_id)

<Experiment: artifact_location='', experiment_id='fe3ff3a4-0fa3-4075-bd4f-8352d3b501d9', lifecycle_stage='active', name='ElasticNet_wine_AML', tags={}>

## 2. Tagging runs

In [27]:
# get the runs
my_experiment
_run = client.get_run(run_id="53254795-c368-4155-a04e-b570ca7d9e60")
print(_run)

<Run: data=<RunData: metrics={'mae': 0.6112547988118586,
 'r2': 0.2157063843066196,
 'rmse': 0.7792546522251949}, params={'alpha': '0.1', 'l1_ratio': '0.1'}, tags={'mlflow.source.git.commit': '5a96d9f2cd8b068a0be1baca9c7c77386f03d047',
 'mlflow.source.name': '/Users/antfra/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/ipykernel_launcher.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'antfra'}>, info=<RunInfo: artifact_uri='azureml://experiments/ElasticNet_wine_AML/runs/53254795-c368-4155-a04e-b570ca7d9e60/artifacts', end_time=1596808619296, experiment_id='fe3ff3a4-0fa3-4075-bd4f-8352d3b501d9', lifecycle_stage='active', run_id='53254795-c368-4155-a04e-b570ca7d9e60', run_uuid='53254795-c368-4155-a04e-b570ca7d9e60', start_time=1596808612524, status='FINISHED', user_id='c8427b67-4103-4ced-b0c1-2200b7c3dcfa'>>


In [14]:
from datetime import datetime
# add a tag to the run
dt = datetime.now().strftime("%d-%m-%Y (%H:%M:%S.%f)")
client.set_tag(_run.info.run_id, "deployed", dt)